In [9]:
import json
import base64
from io import BytesIO
from sagemaker.huggingface.model import HuggingFacePredictor
import datetime
import boto3

ENDPOINT_NAME = 'huggingface-9013840932'

rekognition = boto3.client('rekognition')
predictor = HuggingFacePredictor(endpoint_name=ENDPOINT_NAME, sagemaker_session=sagesession)
translate = boto3.client(service_name='translate')

def get_texts(response):
    response = rekognition.detect_text(Image=image) 
    textDetections=response['TextDetections']
    texts = []
    for text in textDetections:
        texts.append(text['DetectedText'])
    texts = ' '.join(texts)
    return texts

In [4]:
event_body = '...'

In [7]:
current_datetime =  datetime.datetime.isoformat(datetime.datetime.now())
print("current_datetime : {}".format(current_datetime))

input_image = event_body
input_user_id = 'en'
input_language = 'de'
print("input_image : {}".format(input_image))
print("input_user_id : {}".format(input_user_id))
print("input_language : {}".format(input_language))

current_datetime : 2022-12-06T18:45:25.187377
input_image : ...
input_user_id : en
input_language : de


In [10]:
image = {'Bytes': BytesIO(base64.b64decode(input_image)).read()}
response = rekognition.detect_text(Image=image)
extracted_text = get_texts(response)
print("extracted_text : {}".format(extracted_text))


extracted_text : St Lucie County Tribune April 6, 1906 CLEVELAND TO BUILD A HOME AT STUART Lately it has been learned that ex- President Grover Cleveland, who for many winters her been coming south to Stuart, in the northern part of Dade county, has purchased a plot of land in the town and will erect & substantial winter home thereon. The seller WAS Ernest Stypman and the deed will be re- curded in Miami within the next few days. says the Tropical Sun. All the world knows that Grover Cleveland la an enthuniastic fisherman and all the world knows also that Florida St Lucie County Tribune April 6, 1906 CLEVELAND TO BUILD A HOME AT STUART Lately it has been learned that ex- President Grover Cleveland, who for many winters her been coming south to Stuart, in the northern part of Dade county, has purchased a plot of land in the town and will erect & substantial winter home thereon. The seller WAS Ernest Stypman and the deed will be re- curded in Miami within the next few days. says the Trop

In [12]:
response = predictor.predict({
    'inputs': extracted_text
})
original_summary = response[0]['summary_text'].strip()
print("original_summary : {}".format(original_summary))

original_summary : Ex-President Grover Cleveland has purchased a plot of land in Stuart, in the northern part of Dade county, and will erect a substantial winter home thereon . The seller WAS Ernest Stypman and the deed will be re-curded in Miami within the next few days . All the world knows that Grover . Cleveland la an enthuniastic fisherman and all the world . knows also that Florida .


In [13]:
try:
    response = translate.translate_text(Text=original_summary, 
        SourceLanguageCode="en", TargetLanguageCode=input_language)
    summary = response.get('TranslatedText')
    summary_language = input_language
    print("summary : {}".format(summary))
except:
    print("Language {} is not supported. Default to en".format(input_language))
    summary = original_summary
    summary_language = 'en'

summary : Ex-Präsident Grover Cleveland hat ein Grundstück in Stuart im nördlichen Teil von Dade County gekauft und wird darauf ein umfangreiches Winterhaus errichten. Der Verkäufer WAR Ernest Stypman und die Urkunde wird in den nächsten Tagen in Miami erneut verhängt. Die ganze Welt kennt das Grover. Cleveland la ein begeisterter Fischer und die ganze Welt. Weiß auch, dass Florida.


In [14]:
result = {
    'current_datetime': current_datetime,
    'extracted_text': extracted_text,
    'original_summary': original_summary,
    'summary': summary,
    'summary_language': summary_language,
    'input_user_id': input_user_id,
    'input_language': input_language,
}

In [15]:
result

{'current_datetime': '2022-12-06T18:45:25.187377',
 'extracted_text': 'St Lucie County Tribune April 6, 1906 CLEVELAND TO BUILD A HOME AT STUART Lately it has been learned that ex- President Grover Cleveland, who for many winters her been coming south to Stuart, in the northern part of Dade county, has purchased a plot of land in the town and will erect & substantial winter home thereon. The seller WAS Ernest Stypman and the deed will be re- curded in Miami within the next few days. says the Tropical Sun. All the world knows that Grover Cleveland la an enthuniastic fisherman and all the world knows also that Florida St Lucie County Tribune April 6, 1906 CLEVELAND TO BUILD A HOME AT STUART Lately it has been learned that ex- President Grover Cleveland, who for many winters her been coming south to Stuart, in the northern part of Dade county, has purchased a plot of land in the town and will erect & substantial winter home thereon. The seller WAS Ernest Stypman and the deed will be re- c

In [16]:
{
    'statusCode': 200,
    'headers': {
        'Content-Type': 'application/json',
        'Access-Control-Allow-Origin': '*'
    },
    'body': json.dumps(result),
    "isBase64Encoded": False
} 

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Origin': '*'},
 'body': '{"current_datetime": "2022-12-06T18:45:25.187377", "extracted_text": "St Lucie County Tribune April 6, 1906 CLEVELAND TO BUILD A HOME AT STUART Lately it has been learned that ex- President Grover Cleveland, who for many winters her been coming south to Stuart, in the northern part of Dade county, has purchased a plot of land in the town and will erect & substantial winter home thereon. The seller WAS Ernest Stypman and the deed will be re- curded in Miami within the next few days. says the Tropical Sun. All the world knows that Grover Cleveland la an enthuniastic fisherman and all the world knows also that Florida St Lucie County Tribune April 6, 1906 CLEVELAND TO BUILD A HOME AT STUART Lately it has been learned that ex- President Grover Cleveland, who for many winters her been coming south to Stuart, in the northern part of Dade county, has purchased a plot of land i